In [1]:
#Web scraping names and details of stocks with max Gain and Loss using selenium webdriver

In [2]:
#Libraries for webscraping

import selenium
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import time

In [3]:
#setting up browser config for web driver

from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
driverPATH =  r'/home/alby/Downloads/chromedriver'
browserBIN = '/usr/bin/vivaldi'

In [4]:
options = webdriver.ChromeOptions()
options.binary_location = browserBIN

In [5]:
driver = webdriver.Chrome(driverPATH, options=options)

/tmp/ipykernel_33648/2275245732.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driverPATH, options=options)


In [6]:
#accessing url for the site with stock info

url="https://www.bseindia.com/markets.html"
driver.get(url)
time.sleep(5)

In [7]:
#accessing the urls of top 10 gainers and losers

gain_table=driver.find_elements(By.XPATH,"//tbody[@class='ng-scope'and @ng-repeat='G in GainersAndLoosers.Table track by G.scrip_cd']")
links_gain=[]
for j in gain_table:
    time.sleep(2)

    link_tag=j.find_element(By.TAG_NAME,"a")
    links_gain.append(link_tag.get_attribute('href'))

#clicking to access the losers column


links_loss=[]

time.sleep(5)

loss=driver.find_element(By.XPATH,'//*[@id="idLosers"]')
loss.click()

time.sleep(2)

loss_table=driver.find_elements(By.XPATH,"//tbody[@class='ng-scope'and @ng-repeat='L in GainersAndLoosers1.Table track by L.scrip_cd']")


for l in loss_table:

    tag=l.find_element(By.TAG_NAME,"a")
    links_loss.append(tag.get_attribute('href'))





In [8]:
links_loss

['https://www.bseindia.com/stock-share-price/goel-food-products-ltd/goel/543538/',
 'https://www.bseindia.com/stock-share-price/wardwizard-innovations-and-mobility-ltd/wardwizpp/890164/',
 'https://www.bseindia.com/stock-share-price/rita-finance-and-leasing-ltd/rfll/543256/',
 'https://www.bseindia.com/stock-share-price/vikas-proppant--granite-ltd/vikasprop/531518/',
 'https://www.bseindia.com/stock-share-price/aravali-securities--finance-ltd/aravalis/512344/',
 'https://www.bseindia.com/stock-share-price/rmc-switchgears-ltd/rmc/540358/',
 'https://www.bseindia.com/stock-share-price/eki-energy-services-ltd/eki/543284/',
 'https://www.bseindia.com/stock-share-price/arshiya-limited/arshiya/506074/',
 'https://www.bseindia.com/stock-share-price/milestone-furniture-ltd/milefur/541337/',
 'https://www.bseindia.com/stock-share-price/simplex-mills-company-ltd/simplxmil/533018/']

In [9]:
#getting names of top stocks in gain and loss
gain_stocks=[]
loss_stocks=[]
for name in links_gain:
    gain_stocks.append(name.split("/")[4].replace("-"," ").replace("ltd"," "))

#loss stocks
for names in links_loss:
    loss_stocks.append(names.split("/")[4].replace("-"," ").replace("ltd"," "))




In [10]:
gain_stocks,loss_stocks

(['davangere sugar company  ',
  'xchanging solutions  ',
  'bgr energy systems  ',
  'rajeshwari cans  ',
  'gsauto international  ',
  'faze three autofab  ',
  'cmi  ',
  'aki india  ',
  'saboo sodium chloro  ',
  'iti  '],
 ['goel food products  ',
  'wardwizard innovations and mobility  ',
  'rita finance and leasing  ',
  'vikas proppant  granite  ',
  'aravali securities  finance  ',
  'rmc switchgears  ',
  'eki energy services  ',
  'arshiya limited',
  'milestone furniture  ',
  'simplex mills company  '])

In [11]:
###Sns scraping-Twitter for tweets on stocks
tweets = []
ltweets=[]

In [12]:
import snscrape.modules.twitter as sntwitter
import pandas as pd


for gainer in gain_stocks:

    query = str(gainer)+ "lang:en until:2022-07-12 since:2022-06-01"
    
    for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
        tweets.append([tweet.date, tweet.username, tweet.content,gainer,1])

/tmp/ipykernel_33648/3599627893.py:13: FutureWarning: username is deprecated, use user.username instead
  tweets.append([tweet.date, tweet.username, tweet.content,gainer,1])


In [13]:
#Scraping tweets for bottom ten stocks


for loser in loss_stocks:

    query = str(loser)+ "lang:en until:2022-07-12 since:2022-06-01"

    for twee in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
            ltweets.append([twee.date, twee.username, twee.content,loser,0])




/tmp/ipykernel_33648/2621660898.py:12: FutureWarning: username is deprecated, use user.username instead
  ltweets.append([twee.date, twee.username, twee.content,loser,0])


In [14]:
len(tweets)

18331

In [15]:
df1 = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet','company','Stock_status'])

df2 = pd.DataFrame(ltweets, columns=['Date', 'User', 'Tweet','company','Stock_status'])


In [16]:
df2

,Date,User,Tweet,company,Stock_status
0,2022-07-04 10:22:32+00:00,JaayShaan,Stocks touching fresh 52WK-H in today's sessio...,goel food products,0
1,2022-06-28 09:27:03+00:00,VivekIyer72,Goel Food Products Limited becomes the Three H...,goel food products,0
2,2022-06-28 08:25:31+00:00,satinathpk,Goel Food Products.. https://t.co/6HNQfOB3Yq,goel food products,0
3,2022-06-28 06:28:33+00:00,DSIJ,Goel Food Products Limited becomes the Three H...,goel food products,0
4,2022-06-28 05:20:45+00:00,AjayTha26074981,Today 377th SME Goel Food Products LTd got lis...,goel food products,0
...,...,...,...,...,...
123,2022-06-06 21:18:51+00:00,kaymtay,i can’t wait to get my furniture next week 🥹 h...,milestone furniture,0
124,2022-06-06 13:00:11+00:00,CORTFurniture,"50 years ago today CORT got its start, and 50 ...",milestone furniture,0
125,2022-06-06 10:44:24+00:00,EarthMother921,"@TheRealMrsSpit Matching, new, non clearance f...",milestone furniture,0
126,2022-06-04 10:23:34+00:00,amandaamici_,We are excited to share another milestone with...,milestone furniture,0


In [17]:
df1['company'].unique()

array(['davangere sugar company  ', 'xchanging solutions  ',
       'bgr energy systems  ', 'faze three autofab  ', 'cmi  ',
       'aki india  ', 'saboo sodium chloro  ', 'iti  '], dtype=object)